# References
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   - https://research.facebook.com/publications/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks/
 - LangChain documents
   - https://python.langchain.com/docs/modules/chains
 - RAG(Retrieval Augmented Generation)を用いたPostgreSQLアシスタントAIの試作
   - https://qiita.com/comware_hiratsuka/items/7fbc9df423dd64160f73 

# Dependency info

In [ ]:
# To check required dependencies, see `requirements.txt`
# !pip install -r repo/requirements.txt

# Plantform info

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 126
model name	: 06/7e
stepping	: 5
microcode	: 0x1
cpu MHz		: 2303.604
cache size	: 16384 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx rdtscp lm constant_tsc rep_good nopl xtopology cpuid pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch pti fsgsbase bmi1 avx2 smep bmi2 erms avx512f avx512dq rdseed adx smap avx512ifma clflushopt avx512cd sha_ni avx512bw avx512vl xsaveopt xgetbv1 arat avx512vbmi avx512_vpopcntdq rdpid
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit srbds mmio_stale_data retbleed gds
bogomips	: 4607.20
clflush size	: 64
cache_alignment	: 64
address sizes	: 36 bits physic

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            11Gi       438Mi       9.7Gi       5.0Mi       1.6Gi        10Gi
Swap:             0B          0B          0B


# The next cell is used to parse the PostgreSQL documents and generate input documents for external knowledge in RAG
 - A parsing strategy is to parse a table of contents in "index.html" and follow hyperlinks contained there for keeping document structures
   - See `generated/postgres-doc_ja_v15.json` for the output example of this cell
 - TODO
   - Append metadata tags into parsed sentences; for instance, a sentence having SQL commands is attached with the tag that represents its meaning

In [60]:
import bs4
import copy
import json
import re
import requests
import tqdm
from lxml import etree
from pathlib import Path
from typing import Any

# You can change this variable for your input data
# NB: Needs to check if target HTML resources exist in `dataset`
POSTGRES_DOC = 'repo/dataset/postgres-doc_ja_v15'


def collect_html_files_from(ref: str, titles: list[str] = [], files: dict[str, list[str]] = {}) -> dict[str, list[str]]:
    soup = bs4.BeautifulSoup(Path(f"{POSTGRES_DOC}/{ref}").read_text(), 'html.parser')
    
    toc = soup.find("div", {"class": "toc"})
    if toc is None:
        if files.get(ref) is None or len(files[ref]) < len(titles):
            files[ref] = titles

        return files

    if len(titles) == 0 or titles[-1] != title:
        titles = copy.deepcopy(titles)
        titles.append(soup.find('title').text)

    for e in etree.fromstring(str(toc)).iter():
        # Skip redundant links, e.g., xxx.html#yyyy
        if e.tag == 'a' and "#" not in e.get('href'):
            collect_html_files_from(e.get('href'), titles, files)

    return files


def remove_html_tags(s: str) -> str:
    s = re.sub(re.compile('<.*?>'), ' ', s)
    s = re.sub("<!--.+?-->", ' ', s, flags=re.DOTALL)
    return s.strip()


def has_subsect(elem: Any, i: int) -> bool:
    # TODO: Why can't we do it like `elem.findall("div", {"class": f"sect{i+1}"})` here?
    for sect in elem.findall("div"):
        if sect.get('class') == f"sect{i}":
            return True

    return False


def collect_docs(elem: Any, i: int, ref: str, titles: list[str], docs: list[Any]) -> None:
    for sect in elem.findall("div", {"class": f"sect{i}"}):
        if sect.get('class') not in (f"sect{i}", 'chapter', 'refentry', 'appendix', 'bibliography'):
            continue

        current_titles = copy.deepcopy(titles)
        for e in sect.iter():
            if e.tag == f"h{i+1}":
                current_titles.append(remove_html_tags(etree.tostring(e, encoding=str)))

        if has_subsect(sect, i+1):
            collect_docs(sect, i+1, ref, current_titles, docs)
        else:
            content = remove_html_tags(etree.tostring(sect, encoding=str))
            titles_as_str = "[{}]".format(", ".join(map(lambda x: f"\"{x}\"", current_titles)))
            docs.append({"content": content, "ref": ref, "titles": titles_as_str})


def collect_docs_in(ref: str, titles: list[str], docs: list[Any]) -> None:
    soup = bs4.BeautifulSoup(Path(f"{POSTGRES_DOC}/{ref}").read_text(), 'html.parser')
    doc = soup.find("body", {"id": "docContent"})
    root = etree.fromstring(str(doc))
    collect_docs(root, 1, ref, titles, docs)


# A variable to collect input documents for a vector store (e.g., faiss)
docs = []

for ref, titles in tqdm.tqdm(collect_html_files_from('index.html').items()):
    collect_docs_in(ref, titles, docs)

output_docs_filename = f"{POSTGRES_DOC.split('/')[-1]}.json"

with open(output_docs_filename, 'w', encoding='utf-8') as out:
	json.dump(docs, out, indent=2, ensure_ascii=False)

print(f"{len(docs)} documents collected, written into {output_docs_filename}")

100%|██████████| 1054/1054 [00:21<00:00, 49.86it/s]

1897 documents collected, written into postgres-doc_ja_v15.json


## The next cells are used to initialize a vector store with the collected documents above

In [27]:
# Load input documents first
from langchain_community.document_loaders import JSONLoader

# You can change this variable for your input data of a vector store
INPUT_DOCS = 'repo/generated/postgres-doc_ja_v15.json'


def metadata_func(r: dict, md: dict) -> dict:
    md["ref"] = r.get("ref")
    md["titles"] = r.get("titles")
    return md


loader = JSONLoader(file_path=INPUT_DOCS, jq_schema='.[]', content_key='content', metadata_func=metadata_func)
docs = loader.load()

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

# This model has 24 layers, the embedding size is 768, and the maximum # of tokens is 512,
# for more details, see https://huggingface.co/intfloat/multilingual-e5-base
#
# TODO: Replace the model with `intfloat/multilingual-e5-large` for more practical use:
#   - https://huggingface.co/intfloat/multilingual-e5-large
#
# TODO: Try the embeddiing model that has the larger maximum # of tokens, for instance,
# the number in `NousResearch/Yarn-Llama-2-13b-128k` is 128k, see the link below for more details:
#   - https://huggingface.co/NousResearch/Yarn-Llama-2-13b-128k
#
# TODO: Try one of the most popular embedding models, `text-embedding-ada-002`
# (embedding size is 1536 and the maximum # of tokens is 2046) in OpenAI,
# because the PostgreSQL documents are public.
#   - https://platform.openai.com/docs/guides/embeddings
HUGGING_FACE_MODEL_NAME = "intfloat/multilingual-e5-base"
EMBEDDING = HuggingFaceEmbeddings(model_name=HUGGING_FACE_MODEL_NAME)

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [31]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

# Average token length: 2.381185373887978 (name='intfloat/multilingual-e5-base')
def compute_avg_token_length_using_hf_tokenizer(docs: list, name=HUGGING_FACE_MODEL_NAME)-> float:
    tokenizer = AutoTokenizer.from_pretrained(name)
    num_tokens = 0
    length = 0
    for doc in docs:
        tokens = tokenizer.encode(doc.page_content)
        length += len(doc.page_content)
        num_tokens += len(tokens)

    return length / num_tokens


# Average token length: 1.5988751254864615 (name='cl100k_base')
def compute_avg_token_length_usinig_tiktoken(docs: list, name='cl100k_base') -> float:
    import tiktoken
    # cl100k_base: gpt-4, gpt-3.5-turbo, text-embedding-ada-002
    tiktoken_encoding = tiktoken.get_encoding(name)
    num_tokens = 0
    length = 0
    for doc in docs:
        tokens = tiktoken_encoding.encode(doc.page_content)
        length += len(doc.page_content)
        num_tokens += len(tokens)

    return length / num_tokens


# Split page contents in given documents into smaller parts if the size of the contents goes over the limit (`chunk_size`),
# see `generated/postgres-doc_ja_v15_split_doc_example.txt` for a split document example.
#
# A splitting strategy relies on the maximum number of tokens accepted in embedding and chat models.
# In the current implementation, it is 512 in the embedding model and 1024? in the chat model, so
# we set a chunk size at 1000 because we expect that the embedding model could accept
# 512 * 2.38 (averaged chars per token) = ~1218 UTF-8 Japanese characters.
#
# TODO: Tried a two-stage (child & parent) splitter using `ParentDocumentRetriever` and contextual compression
# using `ContextualCompressionRetriever`, but employed a native splitting strategy for now because of simplicity.
# IIUC this strategy highly affects accuracy, so we need to tweak it for smarter splitting.
def split_docs(docs: list, chunk_size: int, chunk_overlap: int) -> list:
    import functools
    splitters = []
    splitters.append(CharacterTextSplitter(separator="\n", chunk_size=chunk_size, chunk_overlap=chunk_overlap))
    splitters.append(RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap))
    docs = functools.reduce(lambda docs, s: s.split_documents(docs), splitters, docs)
    return docs


avg_token_length = compute_avg_token_length_using_hf_tokenizer(docs)
# avg_token_length = compute_avg_token_length_usinig_tiktoken(docs)
print(f"Average token length: {avg_token_length}")

# TODO: Tune the two parameters below, `chunk_size` and `chunk_overlap`
docs = split_docs(docs, chunk_size=1000, chunk_overlap=200)

Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1287, which is longer than the specified 1000
Created a chunk of size 1100, which is longer than the specified 1000
Created a chunk of size 2603, which is longer than the specified 1000
Created a chunk of size 5596, which is longer than the specified 1000
Created a chunk of size 14145, which is longer than the specified 1000
Created a chunk of size 1127, which is longer than the specified 1000
Created a chunk of size 1269, which is longer than the specified 1000
Created a chunk of size 3504, which is longer than the specified 1000
Created a chunk of size 2096, which is longer than the specified 1000
Created a chunk of size 1587, which is longer than the specified 1000
Created a chunk of size 1114, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000


Average token length: 1.5988751254864615


In [31]:
# Appends a header (ref, titles, ...) into the split docs
# TODO: This addition might go over the limit of the context window size of the embedding model
for doc in docs:
    header = []
    header.append("# ref: {}".format(doc.metadata['ref']))
    header.append("# titles: {}".format("/".join(json.loads(doc.metadata['titles']))))
    doc.page_content = "{}\n\n{}".format("\n".join(header), doc.page_content)

In [ ]:
%%timeit
index = FAISS.from_documents(docs, EMBEDDING)

In [45]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as out:
        pickle.dump(obj, out, pickle.HIGHEST_PROTOCOL)


def load_object(filename):
    with open(filename, 'rb') as in_:
        return pickle.load(in_)

In [42]:
index_filename = INPUT_DOCS.split("/")[-1].replace(".json", ".pkl")

In [43]:
# TODO: Probably, we'd be better to use `.save_local()`/`.load_local()` instead?
# pickle stores all the state as is, that is, some confidencial info, e.g., OPENAI_API_KEY

# index.save_local(index_filenam)
save_object(index, index_filename)

In [46]:
# Check if the built vector store works well
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# index = FAISS.load_local("postgresql-doc-15-index", EMBEDDING)
index = load_object(index_filename)

qes = "PostgreSQLが実装しているインデックスの種類は？"
results = index.similarity_search(qes, k=10)
for doc in results:
    print(f"Metadata: {doc.metadata}")

Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 1791, 'ref': 'pgtrgm.html', 'titles': '["PostgreSQL 15.4文書", "F.35. pg_trgm", "F.35.4. インデックスサポート"]'}
Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 1, 'ref': 'intro-whatis.html', 'titles': '["PostgreSQL 15.4文書", "1.  PostgreSQL とは?"]'}
Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 1379, 'ref': 'catalog-pg-index.html', 'titles': '["PostgreSQL 15.4文書", "53.26.  pg_index"]'}
Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 1379, 'ref': 'catalog-pg-index.html', 'titles': '["PostgreSQL 15.4文書", "53.26.  pg_index"]'}
Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 306, 'ref': 'indexes-examine.html', 'titles': '["PostgreSQL 15.4文書", "11.12. インデックス使用状況の検証"]'}
Metadata: {'source': '/home/jovyan/repo/generated/postgres-doc_ja_v15.json', 'seq_num': 3

# The next cells are used to initialize a chat model and do querying for RAG

In [98]:
# TODO: Replace it with an open LLM model, e.g., `elyza/ELYZA-japanese-Llama-2-13b`
#  - https://huggingface.co/elyza/ELYZA-japanese-Llama-2-13b
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

MAX_NUM_TOKENS = 4096

def count_tokens(s: str) -> int:
    def _count_tokens_using_tiktoken(s: str, name='cl100k_base') -> int:
        import tiktoken
        # cl100k_base: gpt-4, gpt-3.5-turbo, text-embedding-ada-002
        return len(tiktoken.get_encoding(name).encode(s))

    return _count_tokens_using_tiktoken(s)


# The maximum number of tokens in 'gpt-3.5-turbo' is 4096, so the number of context docs embedded in a prompt is ~8.
# Newer models in OpenAI accept the larger number of tokens, e.g., one in 'gpt-4-1106-preview' is 128k, so
# the number of embeded docs could increase accordingly.
#
# See a link below for the other models implemented in OpenAI:
#  - https://platform.openai.com/docs/models
OPENAI_MODEL_NAME = 'gpt-3.5-turbo'
# OPENAI_MODEL_NAME = 'gpt-4'
OPENAI_API_KEY = 'YOUR_OPENAI_KEY'

llm = ChatOpenAI(model_name=OPENAI_MODEL_NAME, temperature=0, openai_api_key=OPENAI_API_KEY)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [99]:
# You can change this variable for your indexed documents in a vector store
index = FAISS.load_local('repo/generated/postgresql-doc-15-index', EMBEDDING)

In [118]:
qes = "PostgreSQL 15で実装された新しい機能を箇条書きで可能な限り教えてください"
# qes = "PostgreSQL 15におけるPUBLICスキーマの仕様変更について教えてください"
# qes = "PostgreSQLでクエリの性能が悪いです。どのように調査して改善することができますか"

In [119]:
from langchain import PromptTemplate, LLMChain
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

# Maximum number of context documents embedded in a prompt
MAX_CONTEXT_DOC_NUM = 64

prompt_to_generate_search_words_in_kb = f"""
ユーザからの質問:
{qes}

上記のPostgreSQLに関する質問に回答するためにはナレッジベースを検索して回答する必要があります。
新たな質問を元に、ナレッジベースを検索するための質問文を生成してください。
質問文は日本語で生成してください。
出力内容は質問文のみを含めるようにしてください。
質問文に[]または<<>>で囲まれた文字列を含めないでください。
質問文に'+'のような特殊文字を含めないでください。
質問文をダブルクォーテーションで囲わないでください。
質問文を生成できない場合は、数字の0だけを返してください。
"""

# TODO: Improve search quality with prompt engineering
search_words_in_kb = llm.invoke(system_template).content
docs = index.similarity_search(search_words_in_kb, k=MAX_CONTEXT_DOC_NUM)
# docs = index.similarity_search(qes, k=MAX_CONTEXT_DOC_NUM)

# Tweak context length against the maximum number of LLM tokens
num_context_tokens = count_tokens(template)
context_docs = []
for doc in docs:
    num_tokens = count_tokens(doc.page_content)
    if num_context_tokens + num_tokens > MAX_NUM_TOKENS:
        break
        
    num_context_tokens += num_tokens
    context_docs.append(doc)

context = "\n\n".join([doc.page_content for doc in context_docs])

system_template = f"""
アシスタントはユーザからの質問をサポートします。
詳細な回答を心がけてください。
日本語で回答してください。
以下にリストされた出典に記載されている事実のみを用いて回答してください。
以下の情報が十分でない場合は、分からないと回答してください。
以下に示す関連する出典を用いない回答は作成しないでください。

関連する出典:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    MessagesPlaceholder(variable_name='history'),
    HumanMessagePromptTemplate.from_template('{input}')
])
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=ConversationBufferMemory(memory_key='history', return_messages=True))

print(f"\n========== Context (n={len(context_docs)}, num_tokens={num_context_tokens}) ==========\n")
for i, doc in enumerate(context_docs):
    print("{}: ref={}, titles={}".format(i, doc.metadata['ref'], doc.metadata['titles']))

print("\n========== LLM output ==========\n")
print(llm_chain.run(input=qes))


========== Context (n=6, num_tokens=3538) ==========

0: ref=runtime-config-developer.html, titles=["PostgreSQL 15.4文書", "20.17. 開発者向けのオプション"]
1: ref=runtime-config-developer.html, titles=["PostgreSQL 15.4文書", "20.17. 開発者向けのオプション"]
2: ref=runtime-config-developer.html, titles=["PostgreSQL 15.4文書", "20.17. 開発者向けのオプション"]
3: ref=parallel-plans.html, titles=["PostgreSQL 15.4文書", "15.3. パラレルプラン", "15.3.4. パラレルアペンド"]
4: ref=runtime-config-resource.html, titles=["PostgreSQL 15.4文書", "20.4. 資源の消費", "20.4.6. 非同期動作"]
5: ref=parallel-plans.html, titles=["PostgreSQL 15.4文書", "15.3. パラレルプラン", "15.3.5. パラレルプランに関するヒント"]

========== LLM output ==========



/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


- パラレルプランの改善: パラレルプランの生成と実行の効率が向上しました。また、パラレルプランに関するヒントの使用も可能になりました。
- パラレルアペンドの無効化: パラレルアペンドを無効にするための設定値 `enable_parallel_append` が追加されました。
- 非同期動作の制御: パラレルクエリの実行に関連するリソース消費量を制御するための設定値 `max_parallel_workers_per_gather` が追加されました。
- 開発者向けのオプションの追加: 開発者がテストやデバッグの目的で使用できるオプションが追加されました。例えば、`force_parallel_mode`、`ignore_system_indexes`、`post_auth_delay`、`pre_auth_delay`、`trace_notify`などがあります。
- システムインデックスの無視: システムテーブルの読み込み時にシステムインデックスを無視するための設定値 `ignore_system_indexes` が追加されました。
- デバッグ出力の制御: `trace_notify` を使用して、LISTENとNOTIFYコマンドのデバッグ出力を生成することができます。

以上が、PostgreSQL 15で実装された新しい機能の一部です。詳細な情報は、関連する出典を参照してください。


In [120]:
import pprint
pprint.pprint(llm_chain.memory)

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='PostgreSQL 15で実装された新しい機能を箇条書きで可能な限り教えてください'), AIMessage(content='- パラレルプランの改善: パラレルプランの生成と実行の効率が向上しました。また、パラレルプランに関するヒントの使用も可能になりました。\n- パラレルアペンドの無効化: パラレルアペンドを無効にするための設定値 `enable_parallel_append` が追加されました。\n- 非同期動作の制御: パラレルクエリの実行に関連するリソース消費量を制御するための設定値 `max_parallel_workers_per_gather` が追加されました。\n- 開発者向けのオプションの追加: 開発者がテストやデバッグの目的で使用できるオプションが追加されました。例えば、`force_parallel_mode`、`ignore_system_indexes`、`post_auth_delay`、`pre_auth_delay`、`trace_notify`などがあります。\n- システムインデックスの無視: システムテーブルの読み込み時にシステムインデックスを無視するための設定値 `ignore_system_indexes` が追加されました。\n- デバッグ出力の制御: `trace_notify` を使用して、LISTENとNOTIFYコマンドのデバッグ出力を生成することができます。\n\n以上が、PostgreSQL 15で実装された新しい機能の一部です。詳細な情報は、関連する出典を参照してください。')]), return_messages=True)
